# Compute pRF cortical magnification base on pRF parameters 

In [1]:
# Stop warnings
import warnings
warnings.filterwarnings("ignore")

# Import 
import os 
import time
import numpy as np
import pandas as pd
from scipy import stats
import neuropythy as ny
import matplotlib.pyplot as plt
import scipy.sparse.csgraph as cs


# personal import 
from plot_utils import plotly_template, prf_ecc_pcm_plot
from math_utils import weighted_regression, weighted_nan_percentile, weighted_nan_median



# Apply Wang atlas

In [2]:
def apply_wang(sub, h):
    """Returns the Wang atlas as a property for a subjects hemisphere.

    `apply_wang(sub, 'lh')` applies the atlas to the subject's left hemisphere.
    `apply_wang(sub, 'rh')` applies the atlas to the subject's right hemisphere.
    """
    atl = apply_wang.atlas[h]
    hem = sub.hemis[h]
    fsa = ny.freesurfer_subject(apply_wang.fsaverage_path)
    fsa = fsa.hemis[h]
    return fsa.interpolate(hem, atl)
apply_wang.atlas = {
    h: ny.load(os.path.join(ny.library_path(), 'data', 'fsaverage', 'surf', f'{h}.wang15_mplbl.v1_0.mgz'))
    for h in ('lh', 'rh')}
apply_wang.fsaverage_path = '/home/jovyan/shared/data/freesurfer/fsaverage'

# Geodesic distance (mm)

In [3]:
def compute_geodesic_distance(vert_of_interest_idx, adjacency_matrix, max_distance=None):
    """
    Compute geodesic distances from a vertex to all others in the mesh, optionally limited to max_distance.

    Parameters
    ----------
    vert_of_interest_idx : int
        Index of the vertex from which to compute distances.
    adjacency_matrix : scipy.sparse matrix
        Sparse matrix representing the adjacency of the mesh.
    max_distance : float or None, optional
        If provided, only return distances less than this value.

    Returns
    -------
    median_distance : float
        Median geodesic distance (within mask if max_distance is provided).
    vert_dist_idx : array of int, optional
        Indices of vertices within max_distance (only returned if max_distance is not None).
    """
    distances = cs.dijkstra(adjacency_matrix, indices=[vert_of_interest_idx], directed=False)[0]

    if max_distance is not None:
        distance_mask = distances < max_distance
        vert_dist_idx = np.where(distance_mask)[0]
        median_distance = np.median(distances[distance_mask])
        return median_distance, vert_dist_idx
    else:
        median_distance = np.median(distances)
        return median_distance
        

# Visual distance (dva)

In [4]:
def compute_pRF_distance(vert_of_interest_idx, roi_vertices_hemi_mask, prf_x, prf_y):
    
    dist_array = []
    for vert_num, vert_idx in enumerate(roi_vertices_hemi_mask):
        # Acces pRF position
        target_x = prf_x[vert_of_interest_idx]
        target_y = prf_y[vert_of_interest_idx]
        neigboor_x = prf_x[vert_idx]
        neigboor_y = prf_y[vert_idx]
    
        # Compute visual distance
        vert_prf_dist_array_vert = np.sqrt((target_x - neigboor_x)**2 + (target_y - neigboor_y)**2)
        dist_array.append(vert_prf_dist_array_vert)

    # compute the median of visual distance
    vert_prf_dist_array_median = np.median(dist_array)
    
    return vert_prf_dist_array_median
    

# Compute de pRF CM (mm/dva)

## Acces data

In [5]:
# Settings 
subject_id = 118225
hemis = ['rh', 'lh']
rois = ["V1v", "V1d", "V2v", "V2d", "V3v", "V3d", "hV4", "VO1", "VO2",
          "PHC1", "PHC2", "TO2", "TO1", "LO2", "LO1", "V3B", "V3A",
          "IPS0", "IPS1", "IPS2", "IPS3", "IPS4", "IPS5", "SPL1", "FEF"]
roi_code = {
    "V1v": 1,
    "V1d": 2,
    "V2v": 3,
    "V2d": 4,
    "V3v": 5,
    "V3d": 6,
    "hV4": 7,
    "VO1": 8,
    "VO2": 9,
    "PHC1": 10,
    "PHC2": 11,
    "TO2": 12,
    "TO1": 13,
    "LO2": 14,
    "LO1": 15,
    "V3B": 16,
    "V3A": 17,
    "IPS0": 18,
    "IPS1": 19,
    "IPS2": 20,
    "IPS3": 21,
    "IPS4": 22,
    "IPS5": 23,
    "SPL1": 24,
    "FEF": 25
}

max_ecc = 8
max_distance = 5

In [6]:
# Get first 10 subjects' numbers
allsub = ny.data['hcp_lines'].subject_list
allsub[0:9]

(100610, 118225, 140117, 158136, 172130, 182436, 197348, 214524, 346137)

In [7]:
# Load an HCP subject:
sub = ny.hcp_subject(subject_id)
# data = []
# for ii in allsub[0:9]:
#     sub = ny.hcp_subject(ii)
#     data.append(sub)
sub = ny.data['hcp_lines'].subjects[subject_id]
wang_labels_lh = apply_wang(sub, 'lh')
wang_labels_rh = apply_wang(sub, 'rh')

In [8]:
np.unique(wang_labels_lh)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25])

In [9]:
wang_labels_lh

array([4, 4, 4, ..., 0, 0, 0])

In [10]:
wang_labels_lh.size

157762

In [11]:
wang_labels_rh.size

160031

In [12]:
label_names = (None, "V1v", "V1d", "V2v", "V2d", "V3v", "V3d", "hV4", "VO1", "VO2",
          "PHC1", "PHC2", "TO2", "TO1", "LO2", "LO1", "V3B", "V3A",
          "IPS0", "IPS1", "IPS2", "IPS3", "IPS4", "IPS5", "SPL1", "FEF")

## pRF CM computation

In [18]:
prf_CM_df = pd.DataFrame()
start_time = time.time()

for n_hemi, hemi in enumerate(hemis): 
    for roi in rois: 
        try:
            # ROI mask et paramètres
            if hemi == 'lh':
                label_roi = sub.lh.prop('visual_area')
                prf_x_hemi = sub.lh.prop('prf_x')
                prf_y_hemi = sub.lh.prop('prf_y')
                prf_r2_hemi = sub.lh.prop('prf_variance_explained')
                prf_ecc_hemi = sub.lh.prop('prf_eccentricity')
                white = sub.lh.surface('white') 
            elif hemi == 'rh':
                label_roi = sub.rh.prop('visual_area')
                prf_x_hemi = sub.rh.prop('prf_x')
                prf_y_hemi = sub.rh.prop('prf_y')
                prf_r2_hemi = sub.rh.prop('prf_variance_explained')
                prf_ecc_hemi = sub.rh.prop('prf_eccentricity')
                white = sub.rh.surface('white')

            # Create the ROI mask
            roi_vertices_hemi_mask = np.where(label_roi == roi_code[roi])[0]
            if len(roi_vertices_hemi_mask) == 0:
                raise ValueError(f"No vertices found for ROI {roi} in {hemi}")

            # Access the adjacency_matrix of the ROI
            submesh = white.submesh(roi_vertices_hemi_mask)
            adjacency_matrix = submesh.adjacency_matrix

        except Exception as e:
            print(f"Skipping ROI {roi} in {hemi}: {e}")
            continue  # passe directement à la prochaine ROI
        
        prf_dist_list = [] 
        geo_dist_list = []
        prf_ecc_list = []
        prf_r2_list = []

        for vert_num, vert_idx in enumerate(submesh.labels):
            vert_dist, vert_mask = compute_geodesic_distance(vert_num, adjacency_matrix, max_distance)
            geo_dist_list.append(vert_dist)
            prf_dist_list.append(compute_pRF_distance(
                vert_num, 
                vert_mask, 
                prf_x_hemi[roi_vertices_hemi_mask], 
                prf_y_hemi[roi_vertices_hemi_mask]
            ))
            prf_ecc_list.append(prf_ecc_hemi[vert_idx]) 
            prf_r2_list.append(prf_r2_hemi[vert_idx])
    
        prf_CM_df_hemi = pd.DataFrame({
            'prf_dist': prf_dist_list,
            'geo_dist': geo_dist_list,
            'prf_ecc': prf_ecc_list,
            'prf_r2': prf_r2_list,
            'hemi': hemi,
            'roi': roi,
            'subject': f'{subject_id}'
        })
        
        prf_CM_df = pd.concat([prf_CM_df, prf_CM_df_hemi], ignore_index=True)
    
end_time = time.time()
print("Execution time: {:.2f} seconds".format(end_time - start_time))


Skipping ROI V2d in rh: No vertices found for ROI V2d in rh
Skipping ROI V3v in rh: No vertices found for ROI V3v in rh
Skipping ROI V3d in rh: No vertices found for ROI V3d in rh
Skipping ROI hV4 in rh: No vertices found for ROI hV4 in rh
Skipping ROI VO1 in rh: No vertices found for ROI VO1 in rh
Skipping ROI VO2 in rh: No vertices found for ROI VO2 in rh
Skipping ROI PHC1 in rh: No vertices found for ROI PHC1 in rh
Skipping ROI PHC2 in rh: No vertices found for ROI PHC2 in rh
Skipping ROI TO2 in rh: No vertices found for ROI TO2 in rh
Skipping ROI TO1 in rh: No vertices found for ROI TO1 in rh
Skipping ROI LO2 in rh: No vertices found for ROI LO2 in rh
Skipping ROI LO1 in rh: No vertices found for ROI LO1 in rh
Skipping ROI V3B in rh: No vertices found for ROI V3B in rh
Skipping ROI V3A in rh: No vertices found for ROI V3A in rh
Skipping ROI IPS0 in rh: No vertices found for ROI IPS0 in rh
Skipping ROI IPS1 in rh: No vertices found for ROI IPS1 in rh
Skipping ROI IPS2 in rh: No vert

In [13]:
# prf_CM_df = pd.DataFrame()
# start_time = time.time()

# for n_hemi, hemi in enumerate(hemis): 
#     for roi in rois : 
#         # ROI mask et paramètres
#         if hemi == 'lh':
#             label_roi = sub.lh.prop('visual_area')
#             prf_x_hemi = sub.lh.prop('prf_x')
#             prf_y_hemi = sub.lh.prop('prf_y')
#             prf_r2_hemi = sub.lh.prop('prf_variance_explained')
#             prf_ecc_hemi = sub.lh.prop('prf_eccentricity')
#             white = sub.lh.surface('white') 
#         elif hemi == 'rh':
#             label_roi = sub.rh.prop('visual_area')
#             prf_x_hemi = sub.rh.prop('prf_x')
#             prf_y_hemi = sub.rh.prop('prf_y')
#             prf_r2_hemi = sub.rh.prop('prf_variance_explained')
#             prf_ecc_hemi = sub.rh.prop('prf_eccentricity')
#             white = sub.rh.surface('white')
#         try :
#             # Create the Roi mask
#             roi_vertices_hemi_mask = np.where(label_roi == roi_code[roi])[0]
                    
#             # Acces the adjacency_matrix of the roi
#             submesh = white.submesh(roi_vertices_hemi_mask)
#             adjacency_matrix = submesh.adjacency_matrix
#         execpt 
    
#         prf_dist_list = [] 
#         geo_dist_list = []
#         prf_ecc_list = []
#         prf_r2_list = []

#         for vert_num, vert_idx in enumerate(submesh.labels):
#             vert_dist, vert_mask = compute_geodesic_distance(vert_num, adjacency_matrix, max_distance)
#             geo_dist_list.append(vert_dist)
#             prf_dist_list.append(compute_pRF_distance(vert_num, 
#                                                       vert_mask, 
#                                                       prf_x_hemi[roi_vertices_hemi_mask], 
#                                                       prf_y_hemi[roi_vertices_hemi_mask]))
        
#             prf_ecc_list.append(prf_ecc_hemi[vert_idx]) 
#             prf_r2_list.append(prf_r2_hemi[vert_idx])
    
#         prf_CM_df_hemi = pd.DataFrame()
#         prf_CM_df_hemi['prf_dist'] = prf_dist_list
#         prf_CM_df_hemi['geo_dist'] = geo_dist_list
#         prf_CM_df_hemi['prf_ecc'] = prf_ecc_list
#         prf_CM_df_hemi['prf_r2'] = prf_r2_list
#         prf_CM_df_hemi['hemi'] = hemi  
#         prf_CM_df_hemi['roi'] = roi
#         prf_CM_df_hemi['subject'] = '{}'.format(subject_id)  
        
    
#         prf_CM_df = pd.concat([prf_CM_df, prf_CM_df_hemi], ignore_index=True)
    
# end_time = time.time()
# print("Execution time: {:.2f} seconds".format(end_time - start_time))

ValueError: zero-size array to reduction operation maximum which has no identity

In [8]:
prf_CM_df['pRF_CM'] = prf_CM_df['geo_dist'] / prf_CM_df['prf_dist']

In [9]:
# Export DF 
tsv_dir = '/home/jovyan/projects/pRF-project_NH2025/data/tsv'
os.makedirs(tsv_dir, exist_ok=True)

# tsv_fn = '{}/{}_prf_parameters.tsv'.format(tsv_dir, subject_id)
tsv_fn = '{}/{}_prf_parameters_5mm.tsv'.format(tsv_dir, subject_id)

prf_CM_df.to_csv(tsv_fn, sep="\t", na_rep='NaN', index=False)